# Trying out the geospatial analysis of kerala covid 19 dataset

In [3]:
!pip install geopandas


     |████████████████████████████████| 931kB 7.6MB/s 
     |████████████████████████████████| 14.7MB 300kB/s 
     |████████████████████████████████| 10.9MB 43.3MB/s 


In [0]:
from geopandas.tools import geocode
import pandas as pd

In [5]:
Geo = pd.read_excel("Geospatial.xlsx",columns=['Sl.No','district','LSG','Ward'])
Geo.head()

,Sl. No.,District,LSGs needing special attention,Containment Zones (Ward)
0,1,Palakkad,Akathethara,"2, 3"
1,2,Palakkad,Alanallur,"1, 14, 20"
2,3,Palakkad,Ambalappara,"7, 20"
3,4,Palakkad,Anakkara,"2, 13"
4,5,Palakkad,Chittoor-Thathamangalam,25


In [0]:
Geo.rename(columns={'Sl. No.':'Sl.No',
                          'District':'district',
                          'LSGs needing special attention':'LSG',
                    'Containment Zones (Ward)':'Ward'}, 
                 inplace=True)

In [7]:
Geo.LSG.unique()

array(['Akathethara', 'Alanallur', 'Ambalappara', 'Anakkara',
       'Chittoor-Thathamangalam', 'Cheruppulasherry', 'Elappally',
       'Kannadi', 'Karakurishi', 'Koduvayoor', 'Kollengode', 'Koppam',
       'Kunisherri', 'Lakkidi Peroor', 'Mannarkad (M)', 'Mundoor',
       'Nellay', 'Palakkad (M)', 'Pattambi', 'Pattithara', 'Perumatty',
       'Pirayiri', 'Pookkottukavu', 'Puthupariyaram', 'Puthusherry',
       'Sreekrishnapuram', 'Thachampara', 'Tharoor', 'Thenkara',
       'Thrikkadeeri', 'Vadakkancheri', 'Vallappuzha', 'Vaniyamkulam',
       'Vilayur', 'Shornoor', 'Alakode', 'Anthoor (M)', 'Chembilode',
       'Cherupuzha', 'Chokli', 'Dharmadam', 'Eruvessy', 'Iritty (M)',
       'Kadannappally-Panappuzha', 'Kanichar', 'Kannapuram', 'Kannur (C)',
       'Kottayam Malabar', 'Kuttiattur', 'Mangattidam', 'Mattannur (M)',
       'Mattool', 'Mayyil', 'Munderi', 'Muzhappilangad', 'Muzhakunnu',
       'Naduvil', 'Panniyannur', 'Panoor (M)', 'Pappinissery', 'Pattiam',
       'Payyannur (M)',

In [0]:
Geo['Latitude'] = ''*len(Geo)
Geo['Longitude'] = ''*len(Geo)
Geo['geometry'] = ''*len(Geo)

In [0]:

def my_geocoder(row):
  try:
    point = geocode(row).geometry.iloc[0]
    return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
  except:
    return None
 
Geo[['Latitude','Longitude','geometry']] = Geo.apply(lambda x: my_geocoder(x['LSG']), axis=1)


In [10]:
Geo[['Latitude','Longitude','geometry']]

,Latitude,Longitude,geometry
0,10.832278,76.639961,POINT (76.6399612426125 10.8322782516177)
1,NaN,NaN,None
2,10.831723,76.406776,POINT (76.4067764282125 10.8317232131177)
3,9.666600,77.162567,POINT (77.1625671386126 9.666600227315829)
4,13.219500,79.103302,POINT (79.10330200191289 13.2194995880216)
...,...,...,...
120,NaN,NaN,None
121,9.466670,76.949997,POINT (76.9499969482126 9.46667003631155)
122,NaN,NaN,None
123,31.835560,76.985214,POINT (76.9852142333126 31.8355598449521)


In [0]:
import numpy as np


In [12]:
import geopandas as gpd
print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(Geo["Latitude"])) / len(Geo)) * 100))

# Drop universities that were not successfully geocoded
Geo = Geo.loc[~np.isnan(Geo["Latitude"])]
Geo = gpd.GeoDataFrame(Geo, geometry=Geo.geometry)
Geo.crs = {'init': 'epsg:4326'}
Geo.head()

60.0% of addresses were geocoded!


/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,Sl.No,district,LSG,Ward,Latitude,Longitude,geometry
0,1,Palakkad,Akathethara,"2, 3",10.832278,76.639961,POINT (76.63996 10.83228)
2,3,Palakkad,Ambalappara,"7, 20",10.831723,76.406776,POINT (76.40678 10.83172)
3,4,Palakkad,Anakkara,"2, 13",9.666600,77.162567,POINT (77.16257 9.66660)
4,5,Palakkad,Chittoor-Thathamangalam,25,13.219500,79.103302,POINT (79.10330 13.21950)
5,6,Palakkad,Cheruppulasherry,27,10.876437,76.304207,POINT (76.30421 10.87644)


In [0]:
def split_row(df):
  if ',' in str(df.Ward):
    return str(df.Ward).split(',')
  else:
    return [str(df.Ward)]

Geo['new'] = Geo.apply(split_row,axis=1)

In [17]:
Geo

,Sl.No,district,LSG,Ward,Latitude,Longitude,geometry,new
0,1,Palakkad,Akathethara,"2, 3",10.832278,76.639961,POINT (76.63996 10.83228),"[2, 3]"
2,3,Palakkad,Ambalappara,"7, 20",10.831723,76.406776,POINT (76.40678 10.83172),"[7, 20]"
3,4,Palakkad,Anakkara,"2, 13",9.666600,77.162567,POINT (77.16257 9.66660),"[2, 13]"
4,5,Palakkad,Chittoor-Thathamangalam,25,13.219500,79.103302,POINT (79.10330 13.21950),[25]
5,6,Palakkad,Cheruppulasherry,27,10.876437,76.304207,POINT (76.30421 10.87644),[27]
...,...,...,...,...,...,...,...,...
118,119,Kottayam,Madappally,"5, 9",9.461753,76.590668,POINT (76.59067 9.46175),"[5, 9]"
119,120,Kottayam,Aymanam,3,9.617530,76.509758,POINT (76.50976 9.61753),[3]
121,122,Kottayam,Koruthodu,8,9.466670,76.949997,POINT (76.95000 9.46667),[8]
123,124,Wayanad,Sulthan Bathery (M),"6, 7, 9, 10, 11, 12, 13, 14, 15,16, 22, 23, 24...",31.835560,76.985214,POINT (76.98521 31.83556),"[6, 7, 9, 10, 11, 12, 13, 14, 15, 16, ..."


In [0]:
# convert to list
Geo['Ward'] = Geo['Ward'].str.split(',')

# convert list of pd.Series then stack it
Geo = (Geo
 .set_index(['Sl.No', 'district', 'LSG', 'Latitude', 'Longitude'])['new']
 .apply(pd.Series)
 .stack()
 .reset_index()
 .rename(columns={0:'component'}))


In [19]:
Geo

,Sl.No,district,LSG,Latitude,Longitude,level_5,component
0,1,Palakkad,Akathethara,10.832278,76.639961,0,2
1,1,Palakkad,Akathethara,10.832278,76.639961,1,3
2,3,Palakkad,Ambalappara,10.831723,76.406776,0,7
3,3,Palakkad,Ambalappara,10.831723,76.406776,1,20
4,4,Palakkad,Anakkara,9.666600,77.162567,0,2
...,...,...,...,...,...,...,...
145,124,Wayanad,Sulthan Bathery (M),31.835560,76.985214,10,22
146,124,Wayanad,Sulthan Bathery (M),31.835560,76.985214,11,23
147,124,Wayanad,Sulthan Bathery (M),31.835560,76.985214,12,24
148,124,Wayanad,Sulthan Bathery (M),31.835560,76.985214,13,32


In [0]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import math

In [24]:
# Create the map
m_3 = folium.Map(location=[10.8505, 76.2711], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
mc = MarkerCluster()
for idx, row in Geo.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker([row['Latitude'], row['Longitude']]))
m_3.add_child(mc)

# Display the map
m_3